In [1]:
import pandas as pd
import numpy as np




In [2]:
StopWords = ["i", "me", "my", "myself", "we", "our", "ours", "ourselves", "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", "himself", "she", "her", "hers", "herself", "it", "its", "itself", "they", "them", "their", "theirs", "themselves", "what", "which", "who", "whom", "this", "that", "these", "those", "am", "is", "are", "was", "were", "be", "been", "being", "have", "has", "had", "having", "do", "does", "did", "doing", "a", "an", "the", "and", "but", "if", "or", "because", "as", "until", "while", "of", "at", "by", "for", "with", "about", "against", "between", "into", "through", "during", "before", "after", "above", "below", "to", "from", "up", "down", "in", "out", "on", "off", "over", "under", "again", "further", "then", "once", "here", "there", "when", "where", "why", "how", "all", "any", "both", "each", "few", "more", "most", "other", "some", "such", "no", "nor", "not", "only", "own", "same", "so", "than", "too", "very", "s", "t", "can", "will", "just", "don", "should", "now"]

In [3]:
df = pd.read_csv("AnnotatedData.csv",",")
dfBackup = df
print(len(df))

199994


In [4]:
#Preprocessing
df = df.dropna()
df["headline"] = df["headline"].str.lower()

df['headline'] = df['headline'].str.replace(',', '')
df['headline'] = df['headline'].str.replace(':', '')
df['headline'] = df['headline'].str.replace('.', '')
df['headline'] = df['headline'].str.replace(';', '')
df['headline'] = df['headline'].str.replace("'", '')
df['headline'] = df['headline'].str.replace('?', '')
df['headline'] = df['headline'].str.replace('!', '')
df['headline'] = df['headline'].str.replace('*', '')
df['headline'] = df['headline'].str.replace('(', '')
df['headline'] = df['headline'].str.replace(')', '')
df['headline'] = df['headline'].str.replace('/', '')
df['headline'] = df['headline'].str.replace("`", '')
df['headline'] = df['headline'].str.replace("´", '')
df['headline'] = df['headline'].str.replace("‘", '')
df['headline'] = df['headline'].str.replace("’", '')
df['headline'] = df['headline'].str.replace('\d+', '')

df["headline"] = df["headline"].str.split(pat=' ')

print(df['headline'])


0         [there, were, , mass, shootings, in, texas, la...
1         [will, smith, joins, diplo, and, nicky, jam, f...
2         [hugh, grant, marries, for, the, first, time, ...
3         [jim, carrey, blasts, castrato, adam, schiff, ...
4         [julianna, margulies, uses, donald, trump, poo...
                                ...                        
199989    [couple, buys, entire, jason, wu, for, target,...
199990    [hiv, drugs, not, linked, with, child, psychia...
199991    [smash, star, katharine, mcphee, in, valentino...
199992    [kelly, clarksons, sleek, super, bowl, hair, p...
199993                         [mias, malfunction, was, us]
Name: headline, Length: 199994, dtype: object


In [5]:
#Train  test split

dfTraining=df[:120000]
dfTesting=df[120000:]
TrainingPos = dfTraining[dfTraining["Vader_Label"]=="p"]
TrainingNeg = dfTraining[dfTraining["Vader_Label"]=="n"]
TrainingPos = TrainingPos[:370000]
TrainingNeg = TrainingNeg[:370000]
TestingPos = dfTesting[dfTesting["Vader_Label"]=="p"]
TestingNeg = dfTesting[dfTesting["Vader_Label"]=="n"]
print(len(TrainingPos))
print(len(TrainingNeg))
print(len(TestingPos))
print(len(TestingNeg))

AmountPositiveHeadlines = len(TrainingPos)
AmountNegativeHeadlines = len(TrainingNeg)
length = len(dfTraining)

Vocab = {}

28309
29183
23527
11279


In [6]:
def train_naive_bayes(dfTraining,TrainingNeg,TrainingPos):
    for headline, label in zip(dfTraining["headline"],dfTraining["Vader_Label"]):
        for word in headline:
            if word in Vocab:
                if label == "p":
                    Vocab[word][1] +=  1
                elif label == "n":
                    Vocab[word][0] += 1
            elif word not in Vocab:
                if label == "p":
                    Vocab[word] = [0,1]
                if label == "n":
                    Vocab[word] = [1,0]

    return length, TrainingPos, TrainingNeg, AmountPositiveHeadlines,     AmountNegativeHeadlines, Vocab


In [7]:
length, TrainingPos, TrainingNeg, AmountPositiveHeadlines, AmountNegativeHeadlines, Vocab = train_naive_bayes(dfTraining,TrainingNeg,TrainingPos)

In [8]:
def WordNotZero(word):
    if word not in Vocab:
        return False
    if Vocab[word][0]<=0:
        return False
    if Vocab[word][1]<=0:
        return False
    return True

In [9]:
def test_naive_bayes(dfTesting, length, TrainingPos, TrainingNeg, AmountPositiveHeadlines, AmountNegativeHeadlines, Vocab):
    ClassifiedPos = 0
    ClassifiedNeg = 0
    Korrekt = 0
    Falsch = 0
    p_p = 0
    p_n = 0
    n_n = 0
    n_p = 0
    PositiveNews =[]
    NegativeNews = []
    for headlines,label,index in zip(dfTesting["headline"],dfTesting["Vader_Label"],dfTesting.index):
        PNegTokens = 0.0
        PPosTokens = 0.0
        for word in headlines:
            if WordNotZero(word) == True and (len(word) > 2) and word.isalpha() and word not in StopWords:
                PNegTokens=PNegTokens +np.log(Vocab[word][0]/AmountNegativeHeadlines)
                PPosTokens=PPosTokens+np.log(Vocab[word][1]/AmountPositiveHeadlines)

                PNegTokens=PNegTokens+np.log(AmountNegativeHeadlines/length)
                PPosTokens=PPosTokens+np.log(AmountPositiveHeadlines/length)


        if PNegTokens > PPosTokens:
            
            ClassifiedNeg += 1
            if label == "n":
                NegativeNews.append(index)
                Korrekt += 1
                n_n += 1
            elif label == "p":
                Falsch += 1
                n_p += 1

        elif PNegTokens < PPosTokens:
            ClassifiedPos += 1
            if label == "p":
                PositiveNews.append(index)
                Korrekt += 1
                p_p += 1
                
                
            elif label == "n":
                Falsch += 1
                p_n += 1

    print("Positiv klassifiziert: ",ClassifiedPos)
    print("Negativ klassifiziert: ",ClassifiedNeg)
    print("Korrekt klassifiziert: ",Korrekt)
    print("Falsch klassifiziert: ",Falsch)

    confusion_matrix =[["x", "pos", "neg"],["pos", p_p, p_n],["neg", n_p, n_n]]
    list_as_array = np.array(confusion_matrix)
    print(list_as_array)

    Accuracy =(p_p + n_n)/(p_p +p_n + n_n + n_p)
    print("Accuracy: ",Accuracy)
    return PositiveNews, NegativeNews




In [10]:
PositiveNews, NegativeNews = test_naive_bayes(dfTesting, length, TrainingPos, TrainingNeg, AmountPositiveHeadlines, AmountNegativeHeadlines, Vocab)

Positiv klassifiziert:  55086
Negativ klassifiziert:  24119
Korrekt klassifiziert:  31770
Falsch klassifiziert:  2934
[['x' 'pos' 'neg']
 ['pos' '22331' '1804']
 ['neg' '1130' '9439']]
Accuracy:  0.9154564315352697


In [11]:
print(AmountNegativeHeadlines)
print(AmountPositiveHeadlines)
print(length)
print(np.log(AmountNegativeHeadlines/length))
print(np.log(AmountPositiveHeadlines/length))


29183
28309
120000
-1.4139053948280729
-1.4443119674894642


In [12]:
Positive = {}
for i in PositiveNews:
    Positive[i] = dfBackup.iloc[i]['headline']

for i in Positive:
    print(Positive[i])



 Ourselves, New Study Shows
The Social Network Diet: A Little Help From Your Friends
Elizabeth Smart Wedding: Details Of Smart's Nuptials To Matthew Gilmour
On Divorce, Co-Parenting, Learning To Love Your Ex, And Other Stuff
Oscars 'Best Actress' Winners: 80 Years Of Leading Ladies' Gowns (PHOTOS)
Casa.com: One-Stop Home Shopping Site Launches, Offers Free Shipping
Andrea Creighton, Chado Ralph Rucci Intern: FaceTime, A Beauty Street Style Column
Explaining Perfect
12 American B&Bs Ideal For A Romantic Weekend (PHOTOS)
How To Find The Best Doctors
Drew Barrymore Beauty Evolution (PHOTOS)
Jennifer Love Hewitt's Style Evolution (PHOTOS)
Karlie Kloss vs. Constance Jablonski: Who Wore It Better? (PHOTOS, POLL)
Design Inspiration: Tina We Salute You Cafe Revolving Art Space (PHOTOS)
New York Fashion Week Fall 2012: The Craziest And Coolest Beauty Trends
40:20 Vision: How Yoga Taught Me the True Meaning of Growing Up
Sky, Sea And Smiles In The Philippines (PHOTOS)
Oscars Recipe: 'The Help' C

In [13]:
Negative = {}
for l in NegativeNews:
    Negative[l] = dfBackup.iloc[l]['headline']

for l in Negative:
    print(Negative[l])

Case, Alabama Police Say
Abduction, Assault At Long Island Rail Road Sparks Manhunt (GRAPHIC VIDEO)
George Zimmerman, Neighborhood Watch Captain Who Shot Trayvon Martin, Charged With Violence Before
Know Thine Enemy: The Worst Anti-LGBT Moments Of The Week, March 3 - March 9
Indiana Tornadoes: Federal Disaster Assistance Available, Obama Says
Sea Shepherd Declares Victory: Activists' 'Sabotage' Blamed For Japan Whaling Season Falling Short
Corrections Corporation Of America Sues Florida Town For Blocking New Detention Center
Chicago Takes a Hit From the FDIC as New City Bank Fails With No Acquirer
Greece Debt Crisis: Stimulus From European Union Eyed By Greece
John Mayer Goes On 'Indefinite Hiatus': Singer's Vocal Cord Injury Forces Him To Cancel Tour
Web Giants Face Battle Over 'Do Not Track', Other Consumer Privacy Legislation
The NFL Needs RGIII -- Badly
American Airlines Flight Attendant Rants About Plane Crash, Grounds Flight
Two-Year-Old Throws Tantrum And Gets Kicked Off JetBlue

In [18]:
def ClassifyHeadlines(dfTesting, length, TrainingPos, TrainingNeg, AmountPositiveHeadlines, AmountNegativeHeadlines, Vocab):
    dfHead = pd.read_csv("NewsHeadlines.csv")
    ClassifiedPos = 0
    ClassifiedNeg = 0
    Korrekt = 0
    Falsch = 0
    p_p = 0
    p_n = 0
    n_n = 0
    n_p = 0
    
    for headlines,index in zip(dfHead["title"],dfTesting.index):
        PNegTokens = 0.0
        PPosTokens = 0.0
        for word in headlines:
            if WordNotZero(word) == True and (len(word) > 2) and word.isalpha() and word not in StopWords:
                PNegTokens=PNegTokens +np.log(Vocab[word][0]/AmountNegativeHeadlines)
                PPosTokens=PPosTokens+np.log(Vocab[word][1]/AmountPositiveHeadlines)

                PNegTokens=PNegTokens+np.log(AmountNegativeHeadlines/length)
                PPosTokens=PPosTokens+np.log(AmountPositiveHeadlines/length)


        if PNegTokens > PPosTokens:
            ClassifiedNeg += 1
            NegativeHeadlines.append(index)
 

        elif PNegTokens < PPosTokens:
            ClassifiedPos += 1
            PositiveHeadlines.append(index)

                
                

    print("Positiv klassifiziert: ",ClassifiedPos)
    print("Negativ klassifiziert: ",ClassifiedNeg)


    return PositiveHeadlines, NegativeHeadlines


In [21]:
PositiveHeadlines =[]
NegativeHeadlines = []
PositiveHeadlines, NegativeHeadlines = ClassifyHeadlines(dfTesting, length, TrainingPos, TrainingNeg, AmountPositiveHeadlines, AmountNegativeHeadlines, Vocab)

PositiveUrl = {}
for i in PositiveHeadlines:
    PositiveUrl[i] = dfHead.iloc[i]['title']

for i in PositiveUrl:
    print(PositiveUrl[i])

Positiv klassifiziert:  0
Negativ klassifiziert:  0
